In [10]:
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder

# pd.set_option('display.max_columns', None)
# pd.set_option('display.max_rows', None)

In [11]:
df_preprocessed = pd.read_csv('data/processed/preprocessed.csv.gz', compression='gzip')
df_preprocessed.shape

C:\Users\study\AppData\Local\Temp\ipykernel_4944\3201528848.py:1: DtypeWarning: Columns (5) have mixed types. Specify dtype option on import or set low_memory=False.
  df_preprocessed = pd.read_csv('data/processed/preprocessed.csv.gz', compression='gzip')


(1526659, 123)

Post process steps
* handle outliers
* handling categorical columns (onehot encode or suitable method)
* scaling numerical features
* encoding categorical variables

In [12]:
df_preprocessed.drop(columns='bankacctype_710L', inplace=True)
df_preprocessed['annuitynextmonth_57A'].fillna(0,inplace=True)
df_preprocessed['currdebt_22A'].fillna(0,inplace=True)
df_preprocessed['currdebtcredtyperange_828A'].fillna(0,inplace=True)
df_preprocessed.drop(columns='maxdbddpdtollast6m_4187119P', inplace=True)

df_preprocessed.drop(columns='date_decision', inplace=True)
df_preprocessed['credtype_322L'] = df_preprocessed['credtype_322L'].fillna(df_preprocessed['credtype_322L'].mode()[0])
df_preprocessed['disbursementtype_67L'] = df_preprocessed['disbursementtype_67L'].fillna(df_preprocessed['disbursementtype_67L'].mode()[0])

In [13]:
columns_for_freq_encode = ['lastst_736L','lastapprcommoditycat_1041M','lastapprcommoditytypec_5251766M','lastcancelreason_561M','lastrejectcommoditycat_161M','lastrejectcommodtypec_5251769M','lastrejectreason_759M','lastrejectreasonclient_4145040M','previouscontdistrict_112M']
columns_for_one_hot_encode = ['cardtype_51L','inittransactioncode_186L','credtype_322L','disbursementtype_67L']

In [20]:
len(columns_for_one_hot_encode) + len(columns_for_freq_encode)

13

In [14]:
df_preprocessed.shape

(1526659, 120)

In [17]:
df_preprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,eir_270L,inittransactioncode_186L,...,numinstpaidearly_338L.1,numinstregularpaid_973L.1,numinsttopaygr_769L.1,numinstunpaidmax_3546851L.1,pmtnum_254L.1,posfpd30lastmonth_3976960P.1,price_1097A.1,sumoutstandtotal_3546847A.1,totaldebt_9A.1,totalsettled_863A.1
0,0,201901,0,0,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0
1,1,201901,0,0,NOCARD,0.0,0.0,0.0,0.2999,CASH,...,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
2,2,201901,0,0,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0
3,3,201901,0,0,NOCARD,0.0,0.0,0.0,0.4200,CASH,...,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
4,4,201901,0,1,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0


In [18]:
# separate categorical and numerical cols
numerical_cols = df_preprocessed.select_dtypes(include='number').columns.tolist()
print(len(numerical_cols))
categorical_cols = df_preprocessed.select_dtypes(include='object').columns.tolist()
print(len(categorical_cols))

105
13


In [22]:
numerical_cols

['case_id',
 'MONTH',
 'WEEK_NUM',
 'target',
 'cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'eir_270L',
 'interestrate_311L',
 'lastapprcredamount_781A',
 'lastrejectcredamount_222A',
 'maininc_215A',
 'maxannuity_159A',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstpaidearly_338L',
 'numinstregularpaid_973L',
 'numinsttopaygr_769L',
 'numinstunpaidmax_3546851L',
 'pmtnum_254L',
 'posfpd30lastmonth_3976960P',
 'price_1097A',
 'sumoutstandtotal_3546847A',
 'totaldebt_9A',
 'totalsettled_863A',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m

In [23]:
cols_to_remove = ['case_id', 'MONTH','WEEK_NUM','target']

for value in cols_to_remove:
    numerical_cols.remove(value)

In [24]:
numerical_cols

['cntincpaycont9m_3716944L',
 'cntpmts24_3658933L',
 'commnoinclast6m_3546845L',
 'eir_270L',
 'interestrate_311L',
 'lastapprcredamount_781A',
 'lastrejectcredamount_222A',
 'maininc_215A',
 'maxannuity_159A',
 'numinstls_657L',
 'numinstlsallpaid_934L',
 'numinstpaidearly_338L',
 'numinstregularpaid_973L',
 'numinsttopaygr_769L',
 'numinstunpaidmax_3546851L',
 'pmtnum_254L',
 'posfpd30lastmonth_3976960P',
 'price_1097A',
 'sumoutstandtotal_3546847A',
 'totaldebt_9A',
 'totalsettled_863A',
 'actualdpdtolerance_344P',
 'amtinstpaidbefduel24m_4187115A',
 'annuity_780A',
 'annuitynextmonth_57A',
 'applicationcnt_361L',
 'applications30d_658L',
 'applicationscnt_1086L',
 'applicationscnt_464L',
 'applicationscnt_629L',
 'applicationscnt_867L',
 'avgdbddpdlast24m_3658932P',
 'avgdbddpdlast3m_4187120P',
 'avgdbdtollast24m_4525197P',
 'avgdpdtolclosure24_3658938P',
 'avginstallast24m_3658937A',
 'avgmaxdpdlast9m_3716943P',
 'avgoutstandbalancel6m_4187114A',
 'avgpmtlast12m_4525200A',
 'clien

Outlier Handeling

In [25]:
# use winsorization to handle outliers 
# it clips extream values to specified percentile
def winsorize_outliers(df, column, lower_pct=0.05, upper_pct=0.95):
    lower_bound = df[column].quantile(lower_pct)
    upper_bound = df[column].quantile(upper_pct)
    df[column] = df[column].clip(lower=lower_bound, upper=upper_bound)
    return df

# def winsorize_outsr(df, column, lower_pct=0.05, upper_pct=0.95):
#     lower_bound = daqw
#     df[co2lumn] = df[column].clip(lower=lower_bound, upper=upper_bound)
#     return df


# winsorize num cols
for col in numerical_cols:
    df_preprocessed = winsorize_outliers(df_preprocessed, col)

In [26]:
df_preprocessed.shape

(1526659, 120)

In [27]:
df_preprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L,cntincpaycont9m_3716944L,cntpmts24_3658933L,commnoinclast6m_3546845L,eir_270L,inittransactioncode_186L,...,numinstpaidearly_338L.1,numinstregularpaid_973L.1,numinsttopaygr_769L.1,numinstunpaidmax_3546851L.1,pmtnum_254L.1,posfpd30lastmonth_3976960P.1,price_1097A.1,sumoutstandtotal_3546847A.1,totaldebt_9A.1,totalsettled_863A.1
0,0,201901,0,0,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0
1,1,201901,0,0,NOCARD,0.0,0.0,0.0,0.2999,CASH,...,0.0,0.0,0.0,0.0,18.0,0.0,0.0,0.0,0.0,0.0
2,2,201901,0,0,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,36.0,0.0,0.0,0.0,0.0,0.0
3,3,201901,0,0,NOCARD,0.0,0.0,0.0,0.4200,CASH,...,0.0,0.0,0.0,0.0,12.0,0.0,0.0,0.0,0.0,0.0
4,4,201901,0,1,NOCARD,0.0,0.0,0.0,0.4500,CASH,...,0.0,0.0,0.0,0.0,24.0,0.0,0.0,0.0,0.0,0.0


In [28]:
# fn freq encoding
def frequency_encode(column):
    frequency_map = column.value_counts(normalize=True)
    return column.map(frequency_map)

In [29]:
# one-hot 
one_hot_encoded = pd.get_dummies(df_preprocessed[columns_for_one_hot_encode])

# freq encode
for col in columns_for_freq_encode:
    df_preprocessed[col] = frequency_encode(df_preprocessed[col])

# sclae num
scaler = StandardScaler()
scaled_numerical = scaler.fit_transform(df_preprocessed[numerical_cols])
scaled_numerical_df = pd.DataFrame(scaled_numerical, columns=[f'scaled_{col}' for col in numerical_cols])

In [30]:
# combine all
df_postprocessed = pd.concat([df_preprocessed[['case_id','MONTH','WEEK_NUM','target']], one_hot_encoded, df_preprocessed[columns_for_freq_encode], scaled_numerical_df], axis=1)


In [31]:
df_postprocessed.shape

(1526659, 126)

In [32]:
df_postprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L_INSTANT,cardtype_51L_NOCARD,cardtype_51L_PERSONALIZED,inittransactioncode_186L_CASH,inittransactioncode_186L_NDF,inittransactioncode_186L_POS,...,scaled_numinstpaidearly_338L.1,scaled_numinstregularpaid_973L.1,scaled_numinsttopaygr_769L.1,scaled_numinstunpaidmax_3546851L.1,scaled_pmtnum_254L.1,scaled_posfpd30lastmonth_3976960P.1,scaled_price_1097A.1,scaled_sumoutstandtotal_3546847A.1,scaled_totaldebt_9A.1,scaled_totalsettled_863A.1
0,0,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542
1,1,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.184843,0.0,-1.037614,-0.488194,-0.48857,-0.756542
2,2,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,2.239406,0.0,-1.037614,-0.488194,-0.48857,-0.756542
3,3,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,-0.500012,0.0,-1.037614,-0.488194,-0.48857,-0.756542
4,4,201901,0,1,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542


In [ ]:
# df_postprocessed.to_csv('data/processed/postprocessed_before_outlier_handled.csv.gz', compression='gzip')

In [33]:
df_postprocessed.to_csv('data/processed/postprocessed_after_outlier_handled.csv.gz', compression='gzip')

In [34]:
df_postprocessed.shape

(1526659, 126)

In [35]:
df_postprocessed.head()

,case_id,MONTH,WEEK_NUM,target,cardtype_51L_INSTANT,cardtype_51L_NOCARD,cardtype_51L_PERSONALIZED,inittransactioncode_186L_CASH,inittransactioncode_186L_NDF,inittransactioncode_186L_POS,...,scaled_numinstpaidearly_338L.1,scaled_numinstregularpaid_973L.1,scaled_numinsttopaygr_769L.1,scaled_numinstunpaidmax_3546851L.1,scaled_pmtnum_254L.1,scaled_posfpd30lastmonth_3976960P.1,scaled_price_1097A.1,scaled_sumoutstandtotal_3546847A.1,scaled_totaldebt_9A.1,scaled_totalsettled_863A.1
0,0,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542
1,1,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.184843,0.0,-1.037614,-0.488194,-0.48857,-0.756542
2,2,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,2.239406,0.0,-1.037614,-0.488194,-0.48857,-0.756542
3,3,201901,0,0,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,-0.500012,0.0,-1.037614,-0.488194,-0.48857,-0.756542
4,4,201901,0,1,False,True,False,True,False,False,...,-0.701393,-0.804146,-0.525393,-0.535457,0.869697,0.0,-1.037614,-0.488194,-0.48857,-0.756542
